# NESTS algorithm **Kopuru Vespa Velutina Competition**

Purpose: Bring together weather data, geographic data, food availability data, and identified nests in each municipality of Biscay in order to have a dataset suitable for analysis and potential predictions in a Machine Learning model.

Outputs: QUEENtrain and QUEENpredict datasets *(WBds03_QUEENtrain.csv & WBds03_QUEENpredict.csv)*

@authors:
* mario.bejar@student.ie.edu
* pedro.geirinhas@student.ie.edu
* a.berrizbeitia@student.ie.edu
* pcasaverde@student.ie.edu

## Libraries

In [170]:
import pandas as pd
import numpy as np
import math
from plotnine import *

In [171]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn import preprocessing

## Functions

In [172]:
def silueta(iterations_int, features_df):
    
    silhouettes = []

    for i in range(2,iterations_int,1):
      model = KMeans(n_clusters=i)
      aux = features_df
      model.fit(aux)
      labels = model.labels_
      sol = silhouette_score(aux, labels)
      silhouettes.append(sol)

    silhouette = pd.DataFrame()
    silhouette['Labels'] = silhouettes
    silhouette['NumberOfClusters'] = range(2,iterations_int,1)
    
    return silhouette

In [173]:
def kmedias(numClusters_int, features_df):
    model = KMeans(n_clusters = numClusters_int)

    aux = features_df
    model.fit(aux)
    modelLabels = model.labels_
    modelCenters = model.cluster_centers_

    return pd.Series(modelLabels, index=features_df.index)

In [174]:
def codos(numClusters_int, features_df):
    inertias = []
    for i in range(1,numClusters_int,1):
      model = KMeans(n_clusters=i)
      aux = features_df
      model.fit(aux)
      inertias.append(model.inertia_)

    elbow = pd.DataFrame()
    elbow['Inertia'] = inertias
    elbow['NumberOfClusters'] = range(1,numClusters_int,1)

    return elbow

## Get the data

In [175]:
df01 = pd.read_csv('../../../Input_open_data/ds01_PLANTILLA-RETO-AVISPAS-KOPURU.csv', sep=";")
df02 = pd.read_csv('../../../Input_open_data/ds02_datos-nidos-avispa-asiatica.csv', sep=",")
df03 = pd.read_csv('../../../Input_open_data/ds03_APICULTURA_COLMENAS_KOPURU.csv', sep=";")
df04 = pd.read_csv('../../../Input_open_data/ds04_FRUTALES-DECLARADOS-KOPURU.csv', sep=";")
WBdf01 = pd.read_csv('./WBds01_GEO.csv', sep=',')
WBdf02 = pd.read_csv('./WBds02_METEO.csv', sep=',')
df_population = pd.read_csv('../../../Other_open_data/population.csv', sep=',')
template = pd.read_csv('../../../Input_open_data/ds01_PLANTILLA-RETO-AVISPAS-KOPURU.csv', sep=";")

## Data cleanup

### Getting the names right

In [176]:
# Dropping and Renaming columns in accordance to the DataMap
# DataMap's URL: https://docs.google.com/spreadsheets/d/1Ad7s4IOmj9Tn2WcEOz4ArwedTzDs9Y0_EaUSm6uRHMQ/edit#gid=0

df01.columns = ['municip_code', 'municip_name', 'nests_2020']
df01.drop(columns=['nests_2020'], inplace=True) # just note that this is the final variable to predict in the competition

df02.drop(columns=['JARDUERA_ZENBAKIA/NUM_ACTUACION', 'ERABILTZAILEA_EU/USUARIO_EU', 'ERABILTZAILEA_CAS/USUARIO_CAS', 'HELBIDEA/DIRECCION', 'EGOERA_EU/ESTADO_EU', 'ITXIERA_DATA/FECHA CIERRE', 'ITXIERAKO AGENTEA_EU/AGENTE CIERRE_EU', 'ITXIERAKO AGENTEA_CAS/AGENTE CIERRE_CAS'], inplace=True)
df02.columns = ['waspbust_id', 'year', 'nest_foundDate', 'municip_name', 'species', 'nest_locType', 'nest_hight', 'nest_diameter', 'nest_longitude', 'nest_latitude', 'nest_status']

df03.drop(columns=['CP'], inplace=True)
df03.columns = ['municip_name','municip_code','colonies_amount']

df04.columns = ['agriculture_type','municip_code','municip_name']

In [177]:
# We don't have the "months" specified for any of the records in 2017 ('nest_foundDate' is incorrect for this year), so we'll drop those records
df02 = df02.drop(df02[df02['year'] == 2017].index, inplace = False)

In [178]:
# Cleaning municipality names in ds02 with names from ds01
df02_wrong_mun = ['ABADIÑO' ,'ABANTO Y CIERVANA' ,'ABANTO Y CIERVANA-ABANTO ZIERBENA' ,'AJANGIZ' ,'ALONSOTEGI' ,'AMOREBIETA-ETXANO' ,'AMOROTO' ,'ARAKALDO' ,'ARANTZAZU' ,'AREATZA' ,'ARRANKUDIAGA' ,'ARRATZU' ,'ARRIETA' ,'ARRIGORRIAGA' ,'ARTEA' ,'ARTZENTALES' ,'ATXONDO' ,'AULESTI' ,'BAKIO' ,'BALMASEDA' ,'BARAKALDO' ,'BARRIKA' ,'BASAURI' ,'BEDIA' ,'BERANGO' ,'BERMEO' ,'BERRIATUA' ,'BERRIZ' ,'BUSTURIA' ,'DERIO' ,'DIMA' ,'DURANGO' ,'EA' ,'ELANTXOBE' ,'ELORRIO' ,'ERANDIO' ,'EREÑO' ,'ERMUA' ,'ERRIGOITI' ,'ETXEBARRI' ,'ETXEBARRIA', 'ETXEBARRIa','FORUA' ,'FRUIZ' ,'GALDAKAO' ,'GALDAMES' ,'GAMIZ-FIKA' ,'GARAI' ,'GATIKA' ,'GAUTEGIZ ARTEAGA' ,'GERNIKA-LUMO' ,'GETXO' ,'GETXO ' ,'GIZABURUAGA' ,'GORDEXOLA' ,'GORLIZ' ,'GUEÑES' ,'IBARRANGELU' ,'IGORRE' ,'ISPASTER' ,'IURRETA' ,'IZURTZA' ,'KARRANTZA HARANA/VALLE DE CARRANZA' ,'KARRANTZA HARANA-VALLE DE CARRANZA' ,'KORTEZUBI' ,'LANESTOSA' ,'LARRABETZU' ,'LAUKIZ' ,'LEIOA' ,'LEKEITIO' ,'LEMOA' ,'LEMOIZ' ,'LEZAMA' ,'LOIU' ,'MALLABIA' ,'MAÑARIA' ,'MARKINA-XEMEIN' ,'MARURI-JATABE' ,'MEÑAKA' ,'MENDATA' ,'MENDEXA' ,'MORGA' ,'MUNDAKA' ,'MUNGIA' ,'MUNITIBAR-ARBATZEGI' ,'MUNITIBAR-ARBATZEGI GERRIKAITZ' ,'MURUETA' ,'MUSKIZ' ,'MUXIKA' ,'NABARNIZ' ,'ONDARROA' ,'OROZKO' ,'ORTUELLA' ,'OTXANDIO' ,'PLENTZIA' ,'PORTUGALETE' ,'SANTURTZI' ,'SESTAO' ,'SONDIKA' ,'SOPELA' ,'SOPUERTA' ,'SUKARRIETA' ,'TRUCIOS-TURTZIOZ' ,'UBIDE' ,'UGAO-MIRABALLES' ,'URDULIZ' ,'URDUÑA/ORDUÑA' ,'URDUÑA-ORDUÑA' ,'VALLE DE TRAPAGA' ,'VALLE DE TRAPAGA-TRAPAGARAN' ,'ZALDIBAR' ,'ZALLA' ,'ZAMUDIO' ,'ZARATAMO' ,'ZEANURI' ,'ZEBERIO' ,'ZIERBENA' ,'ZIORTZA-BOLIBAR' ]
df02_correct_mun = ['Abadiño' ,'Abanto y Ciérvana-Abanto Zierbena' ,'Abanto y Ciérvana-Abanto Zierbena' ,'Ajangiz' ,'Alonsotegi' ,'Amorebieta-Etxano' ,'Amoroto' ,'Arakaldo' ,'Arantzazu' ,'Areatza' ,'Arrankudiaga' ,'Arratzu' ,'Arrieta' ,'Arrigorriaga' ,'Artea' ,'Artzentales' ,'Atxondo' ,'Aulesti' ,'Bakio' ,'Balmaseda' ,'Barakaldo' ,'Barrika' ,'Basauri' ,'Bedia' ,'Berango' ,'Bermeo' ,'Berriatua' ,'Berriz' ,'Busturia' ,'Derio' ,'Dima' ,'Durango' ,'Ea' ,'Elantxobe' ,'Elorrio' ,'Erandio' ,'Ereño' ,'Ermua' ,'Errigoiti' ,'Etxebarri' , 'Etxebarria', 'Etxebarria','Forua' ,'Fruiz' ,'Galdakao' ,'Galdames' ,'Gamiz-Fika' ,'Garai' ,'Gatika' ,'Gautegiz Arteaga' ,'Gernika-Lumo' ,'Getxo' ,'Getxo' ,'Gizaburuaga' ,'Gordexola' ,'Gorliz' ,'Güeñes' ,'Ibarrangelu' ,'Igorre' ,'Ispaster' ,'Iurreta' ,'Izurtza' ,'Karrantza Harana/Valle de Carranza' ,'Karrantza Harana/Valle de Carranza' ,'Kortezubi' ,'Lanestosa' ,'Larrabetzu' ,'Laukiz' ,'Leioa' ,'Lekeitio' ,'Lemoa' ,'Lemoiz' ,'Lezama' ,'Loiu' ,'Mallabia' ,'Mañaria' ,'Markina-Xemein' ,'Maruri-Jatabe' ,'Meñaka' ,'Mendata' ,'Mendexa' ,'Morga' ,'Mundaka' ,'Mungia' ,'Munitibar-Arbatzegi Gerrikaitz' ,'Munitibar-Arbatzegi Gerrikaitz' ,'Murueta' ,'Muskiz' ,'Muxika' ,'Nabarniz' ,'Ondarroa' ,'Orozko' ,'Ortuella' ,'Otxandio' ,'Plentzia' ,'Portugalete' ,'Santurtzi' ,'Sestao' ,'Sondika' ,'Sopela' ,'Sopuerta' ,'Sukarrieta' ,'Trucios-Turtzioz' ,'Ubide' ,'Ugao-Miraballes' ,'Urduliz' ,'Urduña/Orduña' ,'Urduña/Orduña' ,'Valle de Trápaga-Trapagaran' ,'Valle de Trápaga-Trapagaran' ,'Zaldibar' ,'Zalla' ,'Zamudio' ,'Zaratamo' ,'Zeanuri' ,'Zeberio' ,'Zierbena' ,'Ziortza-Bolibar',]
df02.municip_name.replace(to_replace = df02_wrong_mun, value = df02_correct_mun, inplace = True)
df02.shape

(6682, 11)

In [179]:
# Translate the `species` variable contents to English
df02.species.replace(to_replace=['AVISPA ASIÁTICA', 'AVISPA COMÚN', 'ABEJA'], value=['Vespa Velutina', 'Common Wasp', 'Wild Bee'], inplace=True)

In [180]:
# Translate the contents of the `nest_locType` and `nest_status` variables to English
# But note that this data makes is of no use from a "forecastoing" standpoint eventually, since we will predict with a one-year offset (and thus, use thigs like weather mostly)

df02.nest_locType.replace(to_replace=['CONSTRUCCIÓN', 'ARBOLADO'], value=['Urban Environment', 'Natural Environment'], inplace=True)
df02.nest_status.replace(to_replace=['CERRADA - ELIMINADO', 'CERRADA - NO ELIMINABLE', 'PENDIENTE DE GRUPO'], value=['Nest Terminated', 'Cannot Terminate', 'Pending classification'], inplace=True)

### Getting the dates right
Including the addition of a `year_offset` variable to comply with the competition's rules

In [181]:
# Changing 'nest_foundDate' the to "datetime" format
df02['nest_foundDate'] = pd.to_datetime(df02['nest_foundDate'])

# Create a "month" variable in the main dataframe
df02['month'] = pd.DatetimeIndex(df02['nest_foundDate']).month

# Create a "year_offset" variable in the main dataframe
# IMPORTANT: THIS REFLECTS OUR ASSUMPTION THAT `YEAR-1` DATA CAN BE USE TO PREDICT `YEAR` DATA, AS MANDATED BY THE COMPETITION'S BASE REQUIREMENTS
df02['year_offset'] = pd.DatetimeIndex(df02['nest_foundDate']).year - 1

In [182]:
df02.columns

Index(['waspbust_id', 'year', 'nest_foundDate', 'municip_name', 'species',
       'nest_locType', 'nest_hight', 'nest_diameter', 'nest_longitude',
       'nest_latitude', 'nest_status', 'month', 'year_offset'],
      dtype='object')

### Creating distinct dataFrames for each `species`

In [183]:
df02.species.value_counts()

Vespa Velutina    6042
Common Wasp        274
Wild Bee           145
Name: species, dtype: int64

In [184]:
df02_vespas = df02.loc[df02.species == 'Vespa Velutina', :]
df02_wasps = df02.loc[df02.species == 'Common Wasp', :]
df02_bees = df02.loc[df02.species == 'Wild Bee', :]

In [185]:
df02_vespas.shape

(6042, 13)

## Create a TEMPLATE dataframe with the missing municipalities and months

In [186]:
template.drop(columns='NIDOS 2020', inplace=True)
template.columns = ['municip_code', 'municip_name']
template['year2019'] = 2019
template['year2018'] = 2018
template['year2017'] = 2017
template = pd.melt(template, id_vars=['municip_code', 'municip_name'], value_vars=['year2019', 'year2018', 'year2017'], value_name = 'year_offset')
template.drop(columns='variable', inplace=True)

In [187]:
for i in range(1,13,1):
    template[i] = i
template = pd.melt(template, id_vars=['municip_code', 'municip_name', 'year_offset'],\
                   value_vars=[1,2,3,4,5,6,7,8,9,10,11,12], value_name = 'month')
template.drop(columns='variable', inplace=True)

In [188]:
template.shape

(4032, 4)

In [189]:
112*12*3 == template.shape[0]

True

In [190]:
template.columns

Index(['municip_code', 'municip_name', 'year_offset', 'month'], dtype='object')

## Merge the datasets

### Match each `municip_name` to its `municip_code` as per the competition's official template (i.e. `df01`)

In [191]:
# Merge dataFrames df01 and df02 by 'municip_name', in order to identify every wasp nest with its 'municip_code'
# The intention is that 'all_the_queens-wasps' will be the final dataFrame to use in the ML model eventually

all_the_queens_wasps = pd.merge(df02_vespas, df01, how = 'left', on = 'municip_name')

In [192]:
# check if there are any municipalities missing from the df02 dataframe, and add them if necessary

df01.municip_code[~df01.municip_code.isin(all_the_queens_wasps.municip_code.unique())]

27    48020
Name: municip_code, dtype: int64

### Input municipalities and months missing from the dataset

In [193]:
all_the_queens_wasps = pd.merge(all_the_queens_wasps, template,\
                                      how = 'outer', left_on = ['municip_code', 'municip_name', 'year_offset', 'month'],\
                                      right_on = ['municip_code', 'municip_name', 'year_offset', 'month'])

In [194]:
all_the_queens_wasps.isnull().sum()

waspbust_id       2394
year              2394
nest_foundDate    2394
municip_name         0
species           2394
nest_locType      2565
nest_hight        2394
nest_diameter     2394
nest_longitude    3298
nest_latitude     3298
nest_status       2394
month                0
year_offset          0
municip_code         0
dtype: int64

In [195]:
all_the_queens_wasps.year.fillna(value='no registers', inplace=True)

In [196]:
all_the_queens_wasps.shape

(8436, 14)

### Discarding some variables
Namely: **species** (since by now they are all Vespa Velutina only), **nest_foundDate**,**nest_longitude**, and **nest_latitude**.

In [197]:
all_the_queens_wasps.drop(columns=['nest_foundDate', 'nest_longitude', 'nest_latitude', 'species'], inplace=True)

### Creating a new categorical variable for Nest Size

[Formula for nest volume](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6723431/)

[Example calculation in cubic meters](https://www.easycalculation.com/shapes/volume-of-prolate-spheroid.php)

In [198]:
#ggplot(aes(x='nest_hight', y='nest_diameter'), all_the_queens_wasps) + geom_point(stat='identity')

In [199]:
#all_the_queens_wasps['nest_volume_l'] = 4/3 * math.pi * (all_the_queens_wasps['nest_hight']/100/2)**2 * (all_the_queens_wasps['nest_diameter']/100/2) * 1000
#all_the_queens_wasps['nest_volume_l'].fillna(0, inplace=True)
all_the_queens_wasps['nest_size'] = all_the_queens_wasps['nest_hight'] * all_the_queens_wasps['nest_diameter']
all_the_queens_wasps['nest_size'].fillna(0, inplace=True)

In [200]:
#all_the_queens_wasps['nest_volume_l'].describe()
all_the_queens_wasps['nest_size'].describe()

count     8436.000000
mean       221.397226
std        940.966217
min          0.000000
25%          0.000000
50%         40.000000
75%        240.000000
max      63750.000000
Name: nest_size, dtype: float64

In [201]:
#vespaVoluminous = all_the_queens_wasps.loc[:, ['municip_code', 'nest_volume_l']].groupby(by='municip_code', as_index=False).mean()
vespaVoluminous = all_the_queens_wasps.loc[:, ['municip_code', 'nest_size']].groupby(by='municip_code', as_index=False).mean()

In [202]:
#ggplot(aes(x='nest_volume_l'), vespaVoluminous) + geom_histogram()
#ggplot(aes(x='nest_size'), vespaVoluminous) + geom_histogram()

In [203]:
#vespaVoluminous['nest_size_equals'] = pd.qcut(vespaVoluminous['nest_size'], 3, labels=['small', 'mid', 'large'])

In [204]:
#vespaVoluminous['nest_size_equals'].value_counts()

In [205]:
#vespaVoluminous['nest_volume_l'] = pd.cut(vespaVoluminous['nest_volume_l'], bins=3, labels=['small', 'mid', 'large'])
vespaVoluminous['nest_size'] = pd.cut(vespaVoluminous['nest_size'], bins=3, labels=['small', 'mid', 'large'])

In [206]:
#vespaVoluminous['nest_volume_l'].value_counts()
vespaVoluminous['nest_size'].value_counts()

small    79
mid      30
large     3
Name: nest_size, dtype: int64

In [207]:
all_the_queens_wasps = pd.merge(all_the_queens_wasps, vespaVoluminous, how = 'left', on= 'municip_code')

In [208]:
#all_the_queens_wasps

### Converting categoricals to dummy variables
... and dropping some variables. Namely: **nest_locType**, **nest_hight**, **nest_diameter**, **nest_size_x**, **nest_size_y**, and **nest_latitude**.

In [209]:
queen_big = pd.get_dummies(all_the_queens_wasps.nest_size_y)
all_the_queens_wasps = pd.concat([all_the_queens_wasps, queen_big], axis=1)

queen_cosmo = pd.get_dummies(all_the_queens_wasps.nest_locType)
all_the_queens_wasps = pd.concat([all_the_queens_wasps, queen_cosmo], axis=1)

queen_hastalavista = pd.get_dummies(all_the_queens_wasps.nest_status)
all_the_queens_wasps = pd.concat([all_the_queens_wasps, queen_hastalavista], axis=1)

all_the_queens_wasps.drop(columns=['nest_locType', 'nest_hight', 'nest_diameter', 'nest_size_y', 'nest_size_x', 'nest_status'], inplace=True)

all_the_queens_wasps.rename(columns = {"small":"fv_size_small", "mid":"fv_size_mid", "large":"fv_size_large",\
                                      "Natural Environment":"fv_type_natural", "Urban Environment":"fv_type_urban",\
                                      "Cannot Terminate":"fv_status_cantkill", "Nest Terminated":"fv_status_dead", "Pending classification":"fv_status_pending"}, inplace = True)

#all_the_queens_wasps

In [210]:
#all_the_queens_wasps.isnull().sum()

### Counting the amount of wasp nests in each municipality, for each month and year, and adding up the rest of dummy categoricals

In [211]:
all_the_queens_wasps = all_the_queens_wasps.loc[:, ['waspbust_id', 'fv_size_small', 'fv_size_mid', 'fv_size_large', 'fv_type_natural', 'fv_type_urban',\
                                                    'fv_status_cantkill', 'fv_status_dead', 'fv_status_pending',\
                                                    'year', 'municip_name', 'municip_code', 'month', 'year_offset']]\
.groupby(by =['year', 'municip_name', 'municip_code', 'month', 'year_offset'], as_index=False)\
.agg({'waspbust_id':'count', 'fv_size_small':'sum', 'fv_size_mid':'sum', 'fv_size_large':'sum', 'fv_type_natural':'sum', 'fv_type_urban':'sum',\
      'fv_status_cantkill':'sum', 'fv_status_dead':'sum', 'fv_status_pending':'sum'})

In [212]:
# let's rename the id to NESTS, now that it has been counted
all_the_queens_wasps.rename(columns = {"waspbust_id":"NESTS"}, inplace = True)

In [213]:
all_the_queens_wasps.columns

Index(['year', 'municip_name', 'municip_code', 'month', 'year_offset', 'NESTS',
       'fv_size_small', 'fv_size_mid', 'fv_size_large', 'fv_type_natural',
       'fv_type_urban', 'fv_status_cantkill', 'fv_status_dead',
       'fv_status_pending'],
      dtype='object')

In [214]:
# for all those "outer merge" rows with no associated year, set their NESTS to zero
all_the_queens_wasps.loc[all_the_queens_wasps.year == 'no registers', ['NESTS']] = 0

In [215]:
all_the_queens_wasps.NESTS.sum() == df02_vespas.shape[0]

True

In [216]:
# verifying that the DataFrame has the right number of rows
all_the_queens_wasps.shape[0] == 112*12*3

True

In [217]:
#all_the_queens_wasps.isnull().sum()

### Food sources

In [218]:
# Group df03 by 'municip_code' because there are multiple rows for each municipality (and we need a 1:1 relationship)
df03 = df03.groupby(by = 'municip_code', as_index= False).colonies_amount.sum()

In [219]:
# Now merge df03 to add number of bee hives (which is a food source for the wasp) in each municipality
# Note that NaNs (unknown amount of hives) are replaced with zeroes for the 'colonies_amount' variable

all_the_queens_wasps = pd.merge(all_the_queens_wasps, df03, how = 'left', on = 'municip_code')
all_the_queens_wasps.colonies_amount.fillna(value=0, inplace=True)

In [220]:
all_the_queens_wasps.shape

(4032, 15)

In [221]:
#all_the_queens_wasps.isnull().sum()

In [222]:
# Group df04 (agricultural food sources) by municipality code, after appending variables with the amount of each type of agricultural product

aux = df04.copy(deep=True)
aux.drop(columns=['municip_name'], inplace=True)

aux['food_fruit'] = np.where(aux['agriculture_type'] == 'FRUTALES', '1', '0')
aux['food_fruit'] = aux['food_fruit'].astype('int')

aux['food_apple'] = np.where(aux['agriculture_type'] == 'MANZANO', '1', '0')
aux['food_apple'] = aux['food_apple'].astype('int')

txakoli_string = df04.agriculture_type[45]
aux['food_txakoli'] = np.where(aux['agriculture_type'] == txakoli_string, '1', '0')
aux['food_txakoli'] = aux['food_txakoli'].astype('int')

aux['food_kiwi'] = np.where(aux['agriculture_type'] == 'AKTINIDIA (KIWI)', '1', '0')
aux['food_kiwi'] = aux['food_kiwi'].astype('int')

aux['food_pear'] = np.where(aux['agriculture_type'] == 'PERAL', '1', '0')
aux['food_pear'] = aux['food_pear'].astype('int')

aux['food_blueberry'] = np.where(aux['agriculture_type'] == 'ARANDANOS', '1', '0')
aux['food_blueberry'] = aux['food_blueberry'].astype('int')

aux['food_raspberry'] = np.where(aux['agriculture_type'] == 'FRAMBUESAS', '1', '0')
aux['food_raspberry'] = aux['food_raspberry'].astype('int')

aux = aux.groupby(by='municip_code', as_index=False).sum()
df04 = aux.copy(deep=True)

In [223]:
# Now merge df04 to add number of each type of food source ('agriculture_type') present in each municipality
# Any municipality not present in df04 will get assigned 'zero' food sources for any given type of fruit

all_the_queens_wasps = pd.merge(all_the_queens_wasps, df04, how = 'left', on= 'municip_code')
all_the_queens_wasps.food_fruit.fillna(value=0, inplace=True)
all_the_queens_wasps.food_apple.fillna(value=0, inplace=True)
all_the_queens_wasps.food_txakoli.fillna(value=0, inplace=True)
all_the_queens_wasps.food_kiwi.fillna(value=0, inplace=True)
all_the_queens_wasps.food_pear.fillna(value=0, inplace=True)
all_the_queens_wasps.food_blueberry.fillna(value=0, inplace=True)
all_the_queens_wasps.food_raspberry.fillna(value=0, inplace=True)

In [224]:
all_the_queens_wasps.shape

(4032, 22)

In [225]:
#all_the_queens_wasps.isnull().sum()

### Geographic
Here, a very important assumption regarding which station corresponds to each municipality is being brought from the HONEYCOMB script

In [226]:
# Adding weather station code to each municipality in all_the_queens_wasps. "No municipality left behind!"
all_the_queens_wasps = pd.merge(all_the_queens_wasps, WBdf01, how = 'left', on= 'municip_code')

In [227]:
all_the_queens_wasps.shape

(4032, 23)

In [228]:
#all_the_queens_wasps.isnull().sum()

In [229]:
all_the_queens_wasps.year_offset.value_counts()

2018    1344
2017    1344
2019    1344
Name: year_offset, dtype: int64

### Weather

MANDATORY ASSUMPTION: As per the competition's rules. 2020 weather data cannot be used to predict 2020's number of wasp nests.

Therefore, **this merge links 2018's wasp nests to 2017's weather data for each corresponding month** (all of which falls under the $2017$ value for `year_offset`).

Likewise, **2019's wasp nests are linked to 2018's weather data for the corresponding month** (all of which falls under the $2018$ value for `year_offset`).

Finally, the $2019$ value for `year_offset` contains zero NESTS and the year 2019's weather which we will use to predict 2020's number of NESTS (the target variable of the competition)

In [230]:
# Now, merge the Main 'all_the_queens_wasps' dataFrame with the weather data 'WBdf02' dataFrame
all_the_queens_wasps = pd.merge(all_the_queens_wasps, WBdf02, how = 'left',\
                                      left_on = ['station_code', 'month', 'year_offset'],\
                                      right_on = ['station_code', 'month', 'year'])

In [231]:
# note that this relabels `year` from the `all_the_queens_wasps` dataframe as `year_x`, and likewise as `year_y` from the WBdf02 dataframe
all_the_queens_wasps.columns

Index(['year_x', 'municip_name', 'municip_code', 'month', 'year_offset',
       'NESTS', 'fv_size_small', 'fv_size_mid', 'fv_size_large',
       'fv_type_natural', 'fv_type_urban', 'fv_status_cantkill',
       'fv_status_dead', 'fv_status_pending', 'colonies_amount', 'food_fruit',
       'food_apple', 'food_txakoli', 'food_kiwi', 'food_pear',
       'food_blueberry', 'food_raspberry', 'station_code', 'index', 'MMM',
       'year_y', 'station_name', 'code_merge', 'merge_cod', 'weath_days_frost',
       'weath_humidity', 'weath_maxLevel', 'weath_midLevel', 'weath_minLevel',
       'weath_days_rain', 'weath_days_rain1mm', 'weath_accuRainfall',
       'weath_10minRainfall', 'weath_1dayRainfall', 'weath_solar',
       'weath_meanTemp', 'weath_maxTemp', 'weath_maxMeanTemp', 'weath_minTemp',
       'weath_meanWindM', 'weath_maxWindM', 'weath_meanDayMaxWind'],
      dtype='object')

In [232]:
all_the_queens_wasps_TRAIN = all_the_queens_wasps.loc[all_the_queens_wasps.year_offset.isin([2017, 2018]), :]
all_the_queens_wasps_PREDICT = all_the_queens_wasps.loc[all_the_queens_wasps.year_offset.isin([2019]), :]

### Adding `Population`, a publicly available dataset

In [233]:
# Adding population by municipality
all_the_queens_wasps_TRAIN = pd.merge(all_the_queens_wasps_TRAIN, df_population, how = 'left',\
                                      left_on= ['municip_code', 'year_offset'],\
                                      right_on = ['municip_code', 'year'])

all_the_queens_wasps_PREDICT = pd.merge(all_the_queens_wasps_PREDICT, df_population, how = 'left',\
                                        left_on= ['municip_code', 'year_offset'],\
                                        right_on = ['municip_code', 'year'])

In [234]:
all_the_queens_wasps_TRAIN.shape

(2688, 49)

In [235]:
all_the_queens_wasps_PREDICT.shape

(1344, 49)

In [236]:
all_the_queens_wasps_PREDICT.shape[0] + all_the_queens_wasps_TRAIN.shape[0] == template.shape[0]

True

## Further cleanup

In [237]:
#dropping unnecessary/duplicate columns
all_the_queens_wasps_TRAIN.drop(columns=['year_y','code_merge', 'merge_cod', 'year_x', 'index', 'MMM'], inplace=True)

In [238]:
all_the_queens_wasps_TRAIN.columns

Index(['municip_name', 'municip_code', 'month', 'year_offset', 'NESTS',
       'fv_size_small', 'fv_size_mid', 'fv_size_large', 'fv_type_natural',
       'fv_type_urban', 'fv_status_cantkill', 'fv_status_dead',
       'fv_status_pending', 'colonies_amount', 'food_fruit', 'food_apple',
       'food_txakoli', 'food_kiwi', 'food_pear', 'food_blueberry',
       'food_raspberry', 'station_code', 'station_name', 'weath_days_frost',
       'weath_humidity', 'weath_maxLevel', 'weath_midLevel', 'weath_minLevel',
       'weath_days_rain', 'weath_days_rain1mm', 'weath_accuRainfall',
       'weath_10minRainfall', 'weath_1dayRainfall', 'weath_solar',
       'weath_meanTemp', 'weath_maxTemp', 'weath_maxMeanTemp', 'weath_minTemp',
       'weath_meanWindM', 'weath_maxWindM', 'weath_meanDayMaxWind', 'year',
       'population'],
      dtype='object')

In [239]:
all_the_queens_wasps_PREDICT.drop(columns=['year_y', 'code_merge', 'merge_cod', 'year_x', 'index', 'MMM'], inplace=True)

In [240]:
all_the_queens_wasps_PREDICT.columns

Index(['municip_name', 'municip_code', 'month', 'year_offset', 'NESTS',
       'fv_size_small', 'fv_size_mid', 'fv_size_large', 'fv_type_natural',
       'fv_type_urban', 'fv_status_cantkill', 'fv_status_dead',
       'fv_status_pending', 'colonies_amount', 'food_fruit', 'food_apple',
       'food_txakoli', 'food_kiwi', 'food_pear', 'food_blueberry',
       'food_raspberry', 'station_code', 'station_name', 'weath_days_frost',
       'weath_humidity', 'weath_maxLevel', 'weath_midLevel', 'weath_minLevel',
       'weath_days_rain', 'weath_days_rain1mm', 'weath_accuRainfall',
       'weath_10minRainfall', 'weath_1dayRainfall', 'weath_solar',
       'weath_meanTemp', 'weath_maxTemp', 'weath_maxMeanTemp', 'weath_minTemp',
       'weath_meanWindM', 'weath_maxWindM', 'weath_meanDayMaxWind', 'year',
       'population'],
      dtype='object')

In [241]:
all_the_queens_wasps_TRAIN.NESTS.sum() == df02_vespas.shape[0]

True

In [242]:
all_the_queens_wasps_PREDICT.NESTS.sum() == 0

True

In [244]:
all_the_queens_wasps_PREDICT

,municip_name,municip_code,month,year_offset,NESTS,fv_size_small,fv_size_mid,fv_size_large,fv_type_natural,fv_type_urban,...,weath_solar,weath_meanTemp,weath_maxTemp,weath_maxMeanTemp,weath_minTemp,weath_meanWindM,weath_maxWindM,weath_meanDayMaxWind,year,population
0,Abadiño,48001,1,2019,0,0,1,0,0,0,...,12.501449,6.1,16.0,8.8,3.4,10.613883,74.702995,40.082916,2019,7599
1,Abadiño,48001,2,2019,0,0,1,0,0,0,...,12.519191,10.0,24.6,16.1,4.7,10.342841,74.387324,40.146571,2019,7599
2,Abadiño,48001,3,2019,0,0,1,0,0,0,...,12.505731,10.6,26.2,16.9,4.6,10.063586,75.072948,40.091499,2019,7599
3,Abadiño,48001,4,2019,0,0,1,0,0,0,...,12.495403,11.7,24.6,17.4,6.4,10.426616,73.978414,40.092840,2019,7599
4,Abadiño,48001,5,2019,0,0,1,0,0,0,...,12.519495,13.1,30.5,19.0,7.6,10.908015,74.612430,40.116476,2019,7599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,Ziortza-Bolibar,48915,8,2019,0,1,0,0,0,0,...,12.528862,19.7,34.3,25.5,14.6,11.573452,74.964378,40.291097,2019,413
1340,Ziortza-Bolibar,48915,9,2019,0,1,0,0,0,0,...,12.558162,17.7,30.2,23.9,12.5,11.595205,75.445151,40.208953,2019,413
1341,Ziortza-Bolibar,48915,10,2019,0,1,0,0,0,0,...,12.561851,15.2,28.1,21.3,10.6,11.666526,76.114760,40.173831,2019,413
1342,Ziortza-Bolibar,48915,11,2019,0,1,0,0,0,0,...,12.520807,10.0,23.6,14.1,6.5,10.706109,76.087953,40.040077,2019,413


## Clustering municipalities

### ... by the size of its Vespa Velutina nests

In [119]:
sizeMatters = all_the_queens_wasps_TRAIN.loc[:, ['municip_code', 'fv_size_small', 'fv_size_mid', 'fv_size_large']].groupby(by='municip_code', as_index=True).mean()

In [120]:
sizeSilhouette = silueta(15, sizeMatters)

In [121]:
#ggplot(aes(x='NumberOfClusters', y='Labels'), sizeSilhouette) + geom_line() + geom_point()

In [122]:
clustersby_size = 5

In [123]:
sizeClusters = pd.DataFrame()
sizeClusters['cluster_size'] = kmedias(clustersby_size, sizeMatters)

In [124]:
sizeClusters['cluster_size'].reset_index()
sizeClusters['cluster_size'].value_counts()

1    81
3    20
0     8
2     2
4     1
Name: cluster_size, dtype: int64

In [125]:
all_the_queens_wasps_TRAIN = pd.merge(all_the_queens_wasps_TRAIN, sizeClusters['cluster_size'], how = 'left', on= 'municip_code')
all_the_queens_wasps_PREDICT = pd.merge(all_the_queens_wasps_PREDICT, sizeClusters['cluster_size'], how = 'left', on= 'municip_code')

### ... by the usual environment of its wasp nests

In [126]:
cosmopolitan = all_the_queens_wasps_TRAIN.loc[:, ['municip_code', 'fv_type_natural', 'fv_type_urban']].groupby(by='municip_code', as_index=True).mean()
cosmoSilhouette = silueta(10, cosmopolitan)
#ggplot(aes(x='NumberOfClusters', y='Labels'), cosmoSilhouette) + geom_line() + geom_point()

In [127]:
clustersby_cosmo = 2

In [128]:
cosmoClusters = pd.DataFrame()
cosmoClusters['cluster_cosmo'] = kmedias(clustersby_cosmo, cosmopolitan)
cosmoClusters['cluster_cosmo'].reset_index()
cosmoClusters['cluster_cosmo'].value_counts()

1    101
0     11
Name: cluster_cosmo, dtype: int64

In [129]:
all_the_queens_wasps_TRAIN = pd.merge(all_the_queens_wasps_TRAIN, cosmoClusters['cluster_cosmo'], how = 'left', on= 'municip_code')
all_the_queens_wasps_PREDICT = pd.merge(all_the_queens_wasps_PREDICT, cosmoClusters['cluster_cosmo'], how = 'left', on= 'municip_code')

### ... by the usual status its wasp nests are left in

In [130]:
survivalists = all_the_queens_wasps_TRAIN.loc[:, ['municip_code', 'fv_status_cantkill', 'fv_status_dead', 'fv_status_pending']].groupby(by='municip_code', as_index=True).mean()
surviveSilhouette = silueta(10, survivalists)
#ggplot(aes(x='NumberOfClusters', y='Labels'), surviveSilhouette) + geom_line() + geom_point()

In [131]:
clustersby_survive = 2

In [132]:
surviveClusters = pd.DataFrame()
surviveClusters['cluster_survive'] = kmedias(clustersby_cosmo, survivalists)
surviveClusters['cluster_survive'].reset_index()
surviveClusters['cluster_survive'].value_counts()

0    100
1     12
Name: cluster_survive, dtype: int64

In [133]:
all_the_queens_wasps_TRAIN = pd.merge(all_the_queens_wasps_TRAIN, surviveClusters['cluster_survive'], how = 'left', on= 'municip_code')
all_the_queens_wasps_PREDICT = pd.merge(all_the_queens_wasps_PREDICT, surviveClusters['cluster_survive'], how = 'left', on= 'municip_code')

### Dropping all that future information (aka, future variables (`fv_...`)) from the dataset

In [134]:
all_the_queens_wasps_TRAIN.drop(columns=['fv_size_small', 'fv_size_mid', 'fv_size_large', 'fv_type_natural', 'fv_type_urban', 'fv_status_cantkill', 'fv_status_dead', 'fv_status_pending'], inplace=True)
all_the_queens_wasps_PREDICT.drop(columns=['fv_size_small', 'fv_size_mid', 'fv_size_large', 'fv_type_natural', 'fv_type_urban', 'fv_status_cantkill', 'fv_status_dead', 'fv_status_pending'], inplace=True)

### ... by the availability of food sources (`food_`)

In [135]:
foodies = all_the_queens_wasps_TRAIN.loc[:, ['municip_code', 'colonies_amount', 'food_fruit', 'food_apple', 'food_txakoli', 'food_kiwi', 'food_pear', 'food_blueberry', 'food_raspberry']].groupby(by='municip_code', as_index=True).mean()
slimSilhouette = silueta(10, foodies)
#ggplot(aes(x='NumberOfClusters', y='Labels'), slimSilhouette) + geom_line() + geom_point()

In [136]:
clustersby_foodie = 2

In [137]:
foodieClusters = pd.DataFrame()
foodieClusters['cluster_food'] = kmedias(clustersby_foodie, foodies)
foodieClusters['cluster_food'].reset_index()
foodieClusters['cluster_food'].value_counts()

0    103
1      9
Name: cluster_food, dtype: int64

In [138]:
all_the_queens_wasps_TRAIN = pd.merge(all_the_queens_wasps_TRAIN, foodieClusters['cluster_food'], how = 'left', on= 'municip_code')
all_the_queens_wasps_PREDICT = pd.merge(all_the_queens_wasps_PREDICT, foodieClusters['cluster_food'], how = 'left', on= 'municip_code')

### Exploring clustering of weather variables (`weath_...`)

#### ... by Humidity-related variables

In [139]:
# scale the dataset using MinMaxScaler, the most common approach

#scalators = ['weath_days_frost', 'weath_humidity', 'weath_maxLevel', 'weath_midLevel', 'weath_minLevel', 'weath_days_rain', 'weath_days_rain1mm', 'weath_accuRainfall', 'weath_10minRainfall', 'weath_1dayRainfall', 'weath_solar', 'weath_meanTemp', 'weath_maxTemp', 'weath_maxMeanTemp', 'weath_minTemp', 'weath_meanWindM', 'weath_maxWindM', 'weath_meanDayMaxWind']
scalators_wet = ['municip_code', 'weath_days_frost', 'weath_humidity', 'weath_days_rain', 'weath_days_rain1mm', 'weath_accuRainfall', 'weath_10minRainfall', 'weath_1dayRainfall', 'weath_solar']

weathercock_water = all_the_queens_wasps_TRAIN[scalators_wet].copy()
weathercock_water.iloc[:,1:] = preprocessing.minmax_scale(weathercock_water.iloc[:,1:])

In [140]:
weathercock_water = weathercock_water.groupby(by='municip_code', as_index=True).mean()
wetSilhouette = silueta(15, weathercock_water)
#ggplot(aes(x='NumberOfClusters', y='Labels'), wetSilhouette) + geom_line() + geom_point()

In [141]:
clustersby_weather_humid = 2

In [142]:
weatherWetClusters = pd.DataFrame()
weatherWetClusters['cluster_weather_wet'] = kmedias(clustersby_weather_humid, weathercock_water)
weatherWetClusters['cluster_weather_wet'].reset_index()
weatherWetClusters['cluster_weather_wet'].value_counts()

0    57
1    55
Name: cluster_weather_wet, dtype: int64

In [143]:
all_the_queens_wasps_TRAIN = pd.merge(all_the_queens_wasps_TRAIN, weatherWetClusters['cluster_weather_wet'], how = 'left', on= 'municip_code')
all_the_queens_wasps_PREDICT = pd.merge(all_the_queens_wasps_PREDICT, weatherWetClusters['cluster_weather_wet'], how = 'left', on= 'municip_code')

#### ... by Temperature-related variables

In [144]:
# scale the dataset using MinMaxScaler, the most common approach

#scalators = ['weath_days_frost', 'weath_humidity', 'weath_maxLevel', 'weath_midLevel', 'weath_minLevel', 'weath_days_rain', 'weath_days_rain1mm', 'weath_accuRainfall', 'weath_10minRainfall', 'weath_1dayRainfall', 'weath_solar', 'weath_meanTemp', 'weath_maxTemp', 'weath_maxMeanTemp', 'weath_minTemp', 'weath_meanWindM', 'weath_maxWindM', 'weath_meanDayMaxWind']
scalators_temp = ['municip_code', 'weath_meanTemp', 'weath_maxTemp', 'weath_maxMeanTemp', 'weath_minTemp']

weathercock_temp = all_the_queens_wasps_TRAIN[scalators_temp].copy()
weathercock_temp.iloc[:,1:] = preprocessing.minmax_scale(weathercock_temp.iloc[:,1:])

In [145]:
weathercock_temp = weathercock_temp.groupby(by='municip_code', as_index=True).mean()
tempSilhouette = silueta(10, weathercock_temp)
#ggplot(aes(x='NumberOfClusters', y='Labels'), tempSilhouette) + geom_line() + geom_point()

In [146]:
clustersby_weather_temp = 2

In [147]:
weatherTempClusters = pd.DataFrame()
weatherTempClusters['cluster_weather_temp'] = kmedias(clustersby_weather_temp, weathercock_temp)
weatherTempClusters['cluster_weather_temp'].reset_index()
weatherTempClusters['cluster_weather_temp'].value_counts()

0    97
1    15
Name: cluster_weather_temp, dtype: int64

In [148]:
all_the_queens_wasps_TRAIN = pd.merge(all_the_queens_wasps_TRAIN, weatherTempClusters['cluster_weather_temp'], how = 'left', on= 'municip_code')
all_the_queens_wasps_PREDICT = pd.merge(all_the_queens_wasps_PREDICT, weatherTempClusters['cluster_weather_temp'], how = 'left', on= 'municip_code')

#### ... by Wind-related variables

In [149]:
# scale the dataset using MinMaxScaler, the most common approach

#scalators = ['weath_days_frost', 'weath_humidity', 'weath_maxLevel', 'weath_midLevel', 'weath_minLevel', 'weath_days_rain', 'weath_days_rain1mm', 'weath_accuRainfall', 'weath_10minRainfall', 'weath_1dayRainfall', 'weath_solar', 'weath_meanTemp', 'weath_maxTemp', 'weath_maxMeanTemp', 'weath_minTemp', 'weath_meanWindM', 'weath_maxWindM', 'weath_meanDayMaxWind']
scalators_wind = ['municip_code', 'weath_meanWindM', 'weath_maxWindM', 'weath_meanDayMaxWind']

weathercock_wind = all_the_queens_wasps_TRAIN[scalators_wind].copy()
weathercock_wind.iloc[:,1:] = preprocessing.minmax_scale(weathercock_wind.iloc[:,1:])

In [150]:
weathercock_wind = weathercock_wind.groupby(by='municip_code', as_index=True).mean()
windSilhouette = silueta(15, weathercock_wind)
#ggplot(aes(x='NumberOfClusters', y='Labels'), windSilhouette) + geom_line() + geom_point()

In [151]:
clustersby_weather_wind = 2

In [152]:
weatherWindClusters = pd.DataFrame()
weatherWindClusters['cluster_weather_wind'] = kmedias(clustersby_weather_wind, weathercock_wind)
weatherWindClusters['cluster_weather_wind'].reset_index()
weatherWindClusters['cluster_weather_wind'].value_counts()

1    91
0    21
Name: cluster_weather_wind, dtype: int64

In [153]:
all_the_queens_wasps_TRAIN = pd.merge(all_the_queens_wasps_TRAIN, weatherWindClusters['cluster_weather_wind'], how = 'left', on= 'municip_code')
all_the_queens_wasps_PREDICT = pd.merge(all_the_queens_wasps_PREDICT, weatherWindClusters['cluster_weather_wind'], how = 'left', on= 'municip_code')

#### ... by Other weather variables

In [154]:
# scale the dataset using MinMaxScaler, the most common approach

#scalators = ['weath_days_frost', 'weath_humidity', 'weath_maxLevel', 'weath_midLevel', 'weath_minLevel', 'weath_days_rain', 'weath_days_rain1mm', 'weath_accuRainfall', 'weath_10minRainfall', 'weath_1dayRainfall', 'weath_solar', 'weath_meanTemp', 'weath_maxTemp', 'weath_maxMeanTemp', 'weath_minTemp', 'weath_meanWindM', 'weath_maxWindM', 'weath_meanDayMaxWind']
scalators_level = ['municip_code', 'weath_maxLevel', 'weath_midLevel', 'weath_minLevel']

weathercock_level = all_the_queens_wasps_TRAIN[scalators_level].copy()
weathercock_level.iloc[:,1:] = preprocessing.minmax_scale(weathercock_level.iloc[:,1:])

In [155]:
weathercock_level = weathercock_level.groupby(by='municip_code', as_index=True).mean()
levelSilhouette = silueta(10, weathercock_level)
#ggplot(aes(x='NumberOfClusters', y='Labels'), slimSilhouette) + geom_line() + geom_point()

In [156]:
clustersby_weather_level = 2

In [157]:
weatherLevelClusters = pd.DataFrame()
weatherLevelClusters['cluster_weather_level'] = kmedias(clustersby_weather_level, weathercock_level)
weatherLevelClusters['cluster_weather_level'].reset_index()
weatherLevelClusters['cluster_weather_level'].value_counts()

0    101
1     11
Name: cluster_weather_level, dtype: int64

In [158]:
all_the_queens_wasps_TRAIN = pd.merge(all_the_queens_wasps_TRAIN, weatherLevelClusters['cluster_weather_level'], how = 'left', on= 'municip_code')
all_the_queens_wasps_PREDICT = pd.merge(all_the_queens_wasps_PREDICT, weatherLevelClusters['cluster_weather_level'], how = 'left', on= 'municip_code')

### Cluster table

In [159]:
#clusters_table = all_the_queens_wasps_TRAIN.groupby('Cluster').mean()
#pd.value_counts(all_the_queens_wasps_TRAIN.Cluster)

In [160]:
#clusters_table.drop(columns=['Id'], inplace=True)
#clusters_table

In [161]:
#clusters_table.to_excel("/Kopuru-VespaVelutina-Clustering.xlsx", index=False)

## Final check

In [162]:
#all_the_queens_wasps_TRAIN.isnull().sum()

In [163]:
#all_the_queens_wasps_PREDICT.isnull().sum()

In [164]:
# check how many rows (municipalities) are there in the dataframe for each year/month combination
pd.crosstab(all_the_queens_wasps.year_offset, all_the_queens_wasps.month)

month,1,2,3,4,5,6,7,8,9,10,11,12
year_offset,,,,,,,,,,,,
2017,112,112,112,112,112,112,112,112,112,112,112,112
2018,112,112,112,112,112,112,112,112,112,112,112,112
2019,112,112,112,112,112,112,112,112,112,112,112,112


In [165]:
# this loops helps verify which municipality may be missing from any given year/month combination
for i in range(1,13,1):
    print(df01.municip_code[~df01.municip_code.isin\
                  (all_the_queens_wasps.loc[(all_the_queens_wasps.month == i) &\
                                            (all_the_queens_wasps.year_offset == 2019),:].\
                   municip_code.unique())])

Series([], Name: municip_code, dtype: int64)
Series([], Name: municip_code, dtype: int64)
Series([], Name: municip_code, dtype: int64)
Series([], Name: municip_code, dtype: int64)
Series([], Name: municip_code, dtype: int64)
Series([], Name: municip_code, dtype: int64)
Series([], Name: municip_code, dtype: int64)
Series([], Name: municip_code, dtype: int64)
Series([], Name: municip_code, dtype: int64)
Series([], Name: municip_code, dtype: int64)
Series([], Name: municip_code, dtype: int64)
Series([], Name: municip_code, dtype: int64)


In [166]:
all_the_queens_wasps_TRAIN.NESTS.sum() == df02_vespas.shape[0]

True

In [167]:
all_the_queens_wasps_PREDICT.NESTS.sum() == 0

True

## Export the TRAINING dataset for the model
A dataset which relates the weather from a previous year (12 months ago) to an amount of NESTS in any given year (and month).

In [168]:
all_the_queens_wasps_TRAIN.to_csv('WBds03_QUEENtrainMONTHS.csv', index=False)

## Export the PREDICTION dataset for the model

In [169]:
all_the_queens_wasps_PREDICT.to_csv('WBds03_QUEENpredictMONTHS.csv', index=False)